# Mounting the drive to this notebook
-  to mount the drive, change the `root_path`

In [1]:
root_path = r'Colab Notebooks/RNN-Captioning/'

In [2]:
 !pip install torchinfo
%load_ext autoreload
%autoreload 2
import os
import sys
from google.colab import drive

drive.mount('/content/drive/',force_remount=True)
PROJECT_PATH  = os.path.join('/content/drive/MyDrive/',root_path)
os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

print('Working directory is: ',os.getcwd())
print('Files in the PROJECT_PATH: ')
!ls '{PROJECT_PATH}'

Mounted at /content/drive/
Working directory is:  /content/drive/MyDrive/Colab Notebooks/RNN-Captioning
Files in the PROJECT_PATH: 
coco  features	model_checkpoints  PreProcessing.ipynb	train
data  model	notebooks	   Testing.ipynb	training.ipynb


In [5]:
from data import *
from model import LSTM
from model import backbone
import torch
device = 'cuda' if torch.cuda.is_available()  else 'cpu'
coco = f'{PROJECT_PATH}coco'
data = f'{PROJECT_PATH}data'
train_features = f'{PROJECT_PATH}features/train'
train_feattures_resnet =  f'{PROJECT_PATH}features/train_resnet'
val_features = f'{PROJECT_PATH}features/val'
val_features_resnet = f'{PROJECT_PATH}features/val_resnet'
model = f'{PROJECT_PATH}model'
train = f'{PROJECT_PATH}train'
train_image = f'/content/coco_images/train2014'
val_image = f'/content/coco_images/val2014'

## Importing the extractor

In [ ]:
from model.backbone import CNNBackbone
cnn = CNNBackbone('r50',attention=True,device=device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 171MB/s]


# Importing the data splits

In [6]:
karpathy_path = f'{coco}/dataset_coco.json'
splitter = KarpathySplit(karpathy_path)
train,val,test = splitter.get_splits()

In [ ]:
len(val) #first five data points in the split

5000

In [ ]:
vocab = Vocabulary(min_freq=2)
vocab.build(train)

In [ ]:
vocab.__len__()

7931

In [ ]:
vocab_path = f'{PROJECT_PATH}/coco/vocab03.json'
vocab.save(vocab_path)

In [ ]:
enc_train = vocab.encode_captions(train)
enc_val = vocab.encode_captions(val)

## Making separate lists of image_names to be precomputed for training and validation

In [ ]:
train_precomputation = []
val_precomputation = []
for dicts in enc_train:
  train_precomputation.append((dicts['path'],dicts['id']))
for dicts in enc_val:
  val_precomputation.append((dicts['path'],dicts['id']))


## Saving the Data Splits along with Vocabulary in Local Drive Storage (available in the `project/coco` folder) - NOT TO BE RUN IF IMPORTING THE FEATURES

In [ ]:
import json
with open(f'{PROJECT_PATH}coco/train_images.json','w') as f:
  json.dump(train_precomputation,f)
with open(f'{PROJECT_PATH}coco/val_images.json','w') as f:
  json.dump(val_precomputation,f)

In [ ]:
import json
with open(f'{PROJECT_PATH}coco/train2.json','w') as f:
  json.dump(enc_train,f)
with open(f'{PROJECT_PATH}coco/val2.json','w') as f:
  json.dump(enc_val,f)
with open(f'{PROJECT_PATH}coco/test2.json','w') as f:
  json.dump(test,f)

# Download the coco image in VM Storage
--------
- this step happens once, after that the data splits are stored in the coco folder
- from where they can be loaded, whenever needed

In [ ]:
downloader = CocoDownloader('/content/coco_images')
downloader.run()

Extracting train...
train downloaded and extracted.


Extracting val...
val downloaded and extracted.


In [ ]:
import shutil
import os
def move_test_images(image_root,dst_path,test_samples):
  for samples in test_samples:
    filename = samples['path']
    src_path = os.path.join(image_root,filename)
    shutil.move(src_path, dst_path)
    print(f"File '{src_path}' moved successfully to '{os.path.join(dst_path,filename)}'")


## Moving test images to test folder in coco

In [ ]:
test_sample = f'{coco}test-images'

move_test_images('/content/coco_images/val2014',test_sample,test)

File '/content/coco_images/val2014/COCO_val2014_000000375484.jpg' moved successfully to '/content/drive/MyDrive/Colab Notebooks/RNN-Captioning/coco/test-images/COCO_val2014_000000375484.jpg'
File '/content/coco_images/val2014/COCO_val2014_000000038938.jpg' moved successfully to '/content/drive/MyDrive/Colab Notebooks/RNN-Captioning/coco/test-images/COCO_val2014_000000038938.jpg'
File '/content/coco_images/val2014/COCO_val2014_000000487076.jpg' moved successfully to '/content/drive/MyDrive/Colab Notebooks/RNN-Captioning/coco/test-images/COCO_val2014_000000487076.jpg'
File '/content/coco_images/val2014/COCO_val2014_000000276018.jpg' moved successfully to '/content/drive/MyDrive/Colab Notebooks/RNN-Captioning/coco/test-images/COCO_val2014_000000276018.jpg'
File '/content/coco_images/val2014/COCO_val2014_000000485731.jpg' moved successfully to '/content/drive/MyDrive/Colab Notebooks/RNN-Captioning/coco/test-images/COCO_val2014_000000485731.jpg'
File '/content/coco_images/val2014/COCO_val20

# Saving the precomputed feature maps in local storage
-------
- eventually, this will be the only data accessed by the model during training
- each feature map, will be stored with 'img_id.pt' where img_id is the corresponding id of each image in the Karpathy Split used earlier for data split making

# FEATURES ARE ALREADY PROVIDED IN THE PROJECT UNDER features FOLDER, ONLY RUN WHEN MAKING NEW FEATURES

In [ ]:
from data.loaders import GetData
from data.transform import get_transform

In [ ]:
gd_train = GetData(train_feattures_resnet,cnn,get_transform('r50'),train_image,train_precomputation)
gd_val = GetData(val_features_resnet,cnn,get_transform('r50'),val_image,val_precomputation)

In [ ]:
gd_train.batched_pre_compute()
gd_val.batched_pre_compute()

  0%|          | 0/313 [00:00<?, ?it/s]

Saved 20000 features to /content/drive/MyDrive/Colab Notebooks/RNN-Captioning/features/train_resnet


  0%|          | 0/79 [00:00<?, ?it/s]

Saved 5000 features to /content/drive/MyDrive/Colab Notebooks/RNN-Captioning/features/val_resnet
